In [ ]:
# SA

import math
import random
import matplotlib.pyplot as plt
import time

def load_dataset(file_path):
    cities = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[6:-1]:
            parts = line.split()
            city_number = int(parts[0])
            x, y = float(parts[1]), float(parts[2])
            cities.append((city_number, x, y))
    return cities

def euclidean_distance(city1, city2):
    return math.sqrt((city1[1] - city2[1])**2 + (city1[2] - city2[2])**2)

def total_distance(cities, tour):
    distance = 0
    for i in range(len(tour) - 1):
        distance += euclidean_distance(cities[tour[i]], cities[tour[i + 1]])
    distance += euclidean_distance(cities[tour[-1]], cities[tour[0]])
    return distance

def two_opt_swap(tour, i, j):
    return tour[:i] + tour[i:j+1][::-1] + tour[j+1:]

def simulated_annealing(cities, initial_temp=200000, cooling_rate=0.9995, stopping_temp=1e-8, max_iter=500000):
    n = len(cities)
    current_tour = list(range(n))
    random.shuffle(current_tour)
    current_distance = total_distance(cities, current_tour)
    best_tour = current_tour[:]
    best_distance = current_distance

    temperature = initial_temp
    iter_count = 0

    while temperature > stopping_temp and iter_count < max_iter:
        i, j = sorted(random.sample(range(n), 2))
        new_tour = two_opt_swap(current_tour, i, j)
        new_distance = total_distance(cities, new_tour)

        delta = new_distance - current_distance
        if delta < 0 or random.random() < math.exp(-delta / temperature):
            current_tour = new_tour
            current_distance = new_distance
            if new_distance < best_distance:
                best_tour = new_tour
                best_distance = new_distance

        temperature *= cooling_rate
        iter_count += 1

    return best_tour, best_distance

def plot(cities, tour):
    tour_cities = [cities[i] for i in tour]
    tour_cities.append(tour_cities[0])
    x, y = zip(*[(city[1], city[2]) for city in tour_cities])
    plt.plot(x, y, 'o-', markersize=4)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Best Tour')
    plt.show()

    print("Optimal Path (City Numbers):", ' -> '.join(str(cities[i][0]) for i in tour) + ' -> ' + str(cities[tour[0]][0]))
    print("Total Distance (Optimal Path):", total_distance(cities, tour))

start_time = time.time()

cities = load_dataset("kroB200.txt")
best_tour, best_distance = simulated_annealing(cities)
plot(cities, best_tour)

end_time = time.time()

execution_time = end_time - start_time
print(f"Execution Time: {execution_time:.4f} seconds")


In [ ]:
# SA (deterministic)

import math
import matplotlib.pyplot as plt
import time

def load_dataset(file_path):
    cities = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[6:-1]:
            parts = line.split()
            city_number = int(parts[0])
            x, y = float(parts[1]), float(parts[2])
            cities.append((city_number, x, y))
    return cities

def euclidean_distance(city1, city2):
    return math.sqrt((city1[1] - city2[1])**2 + (city1[2] - city2[2])**2)

def total_distance(cities, tour):
    distance = 0
    for i in range(len(tour) - 1):
        distance += euclidean_distance(cities[tour[i]], cities[tour[i + 1]])
    distance += euclidean_distance(cities[tour[-1]], cities[tour[0]])
    return distance

def two_opt_swap(tour, i, j):
    return tour[:i] + tour[i:j+1][::-1] + tour[j+1:]

def deterministic_sa(cities, initial_temp=200000, cooling_rate=0.9995, stopping_temp=1e-8, max_iter=500000):
    n = len(cities)

    # مسیر اولیه قطعی
    current_tour = list(range(n))
    current_distance = total_distance(cities, current_tour)

    best_tour = current_tour[:]
    best_distance = current_distance

    temperature = initial_temp
    iter_count = 0

    while temperature > stopping_temp and iter_count < max_iter:

        # انتخاب قطعی i و j
        i = iter_count % (n - 1)
        j = i + 1

        # ساخت مسیر جدید
        new_tour = two_opt_swap(current_tour, i, j)
        new_distance = total_distance(cities, new_tour)

        delta = new_distance - current_distance

        # فقط جواب بهتر پذیرفته می‌شود
        if delta < 0:
            current_tour = new_tour
            current_distance = new_distance

            if new_distance < best_distance:
                best_tour = new_tour
                best_distance = new_distance

        # کاهش دما
        temperature *= cooling_rate
        iter_count += 1

    return best_tour, best_distance

def plot(cities, tour):
    tour_cities = [cities[i] for i in tour]
    tour_cities.append(tour_cities[0])
    x, y = zip(*[(city[1], city[2]) for city in tour_cities])
    plt.plot(x, y, 'o-', markersize=4)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Deterministic Tour')
    plt.show()

    print("Optimal Path (City Numbers):", ' -> '.join(str(cities[i][0]) for i in tour) + ' -> ' + str(cities[tour[0]][0]))
    print("Total Distance:", total_distance(cities, tour))


# اجرای نسخه قطعی
start = time.time()

cities = load_dataset("kroB200.txt")   # :contentReference[oaicite:0]{index=0}
best_tour, best_distance = deterministic_sa(cities)
plot(cities, best_tour)

end = time.time()
execution_time = end - start

print(f"Execution Time: {execution_time:.4f} seconds")
